# Math 425 Fall 2025 Project 1  
**Due:** 5PM on Fri October 10  

## Project Directions
- Include a report on every group member’s contribution.  
- Submit the group’s well-commented code used for the project with instructions on how to compile and run.  
- Make a 15 to 20 minute video presentation of your results.  

The project consists of 3 problems.

You are given part of the **Wisconsin Diagnostic Breast Cancer (WDBC) dataset**. For each patient, you are given a vector $\mathbf{a}$ giving features computed from digitized images of a fine needle aspirate (FNA) of a breast mass. The features describe characteristics of the cell nuclei present in the image. The goal is to decide whether the cells are malignant or benign.  

### Feature Computation
Ten real-valued quantities are computed for each cell nucleus:
- **radius** (mean of distances from center to points on the perimeter)  
- **texture** (standard deviation of gray-scale values)  
- **perimeter**  
- **area**  
- **smoothness** (local variation in radius lengths)  
- **compactness** $\;=\; \dfrac{\text{perimeter}^2}{\text{area}} - 1.0$  
- **concavity** (severity of concave portions of the contour)  
- **concave points** (number of concave portions of the contour)  
- **symmetry**  
- **fractal dimension** (“coastline approximation” - 1)  

The **mean**, **standard error (stderr)**, and a measure of the **largest (worst)** values were computed for each feature.  
Thus, each specimen is represented by a vector $\mathbf{a}$ with **30 entries**.  

The domain $D$ consists of thirty strings identifying these features, e.g.  
- `radius (mean)`  
- `radius (stderr)`  
- `radius (worst)`  
- `area (mean)`  

### Provided Files
- `train.txt`: data for 300 patients  
- `train_values.txt`: indicator for malignant specimen (+1) or benign specimen (−1)  
- `validate.txt`: data for 260 patients  
- `validate_values.txt`: indicator for malignant specimen (+1) or benign specimen (−1) 

## Problem 1
**(a)** Apply $k$-means clustering with $k=2$ to the training data.  
Then use the validation data to assess clustering accuracy. You will need a scheme to determine whether a patient in the validation set has a malignant or benign tumor based on clustering.  

**(b)** Embed the data in dimensions $d \in \{5, 10, 20\}$ using **Gaussian matrix embedding**, then rerun $k$-means.  
- What is the clustering accuracy for each $d$?  
- What is the computational time averaged over 500 independent runs? 

**(c)** Repeat part (b) but use **sparse random projection** instead of Gaussian embedding.  

## Problem 2
- Read the data in `train.txt` into a matrix $A$ (rows = patients, columns = 30 features).  
- Read the data in `train_values.txt` into a vector $b$ where  

  $$
  b_i = \begin{cases} 
  +1 & \text{if malignant} \\ 
  -1 & \text{if benign} 
  \end{cases}
  $$

**(a)** Use the **QR algorithm** to find the least-squares linear model for the data. 

**(b)** Apply the linear model to `validate.txt` and predict malignancy. Define a classifier:

$$
C(y) = \begin{cases} 
+1 & \text{if } y \geq 0 \\ 
-1 & \text{otherwise}
\end{cases}
$$

**(c)** What is the percentage of incorrectly classified samples? Compare with the success rate on the training data.  

**(d)** Embed the data in $d \in \{5, 10, 20\}$ using **Gaussian matrix embedding** and repeat (a), (b), and (c). Report average computational time over 500 runs.  

**(e)** Repeat part (d) but use **sparse random projection** instead.  

## Problem 3
Apply $k$-means to the **class music data** `songList.xlsx` and use **Class Roster** to group the class into **8 distinct music clusters**.
